In [ ]:
%matplotlib inline

import os
import sys
import certifi
import unicodedata
import pandas as pd
import numpy as np
import re
import csv
import math
from datetime import datetime
import requests
import time
import scipy.stats as stats
import matplotlib.pyplot as plt

from collections import defaultdict
from scipy.stats.stats import pearsonr

# DATA PRE-PROCESSING
Load csv file with answer and question. Create a new column with answer+question and drop column id and tag

In [ ]:
df = pd.read_csv("Dataset/faqs_uft8_QA.csv", sep = ";", encoding = 'utf8', quotechar = '"')
df.head()

In [ ]:
def substring(line):
    car=line[-1]
    if car!='.':
        line=line+'.'
    return line

In [ ]:
df['question'] = df['question'].map(lambda x:x.replace('?','').replace('"','').replace('. ',''))
df['answer'] = df['answer'].map(lambda x:x.replace('. ',' ').replace('"','').replace('?',''))

df['QA'] = df['question'] + ' ' + df['answer']
df['Y'] = 1

df['QA'] = df['QA'].map(lambda x:x.replace('?','').replace('"','').replace('. ','').replace('. ',''))
df['QA'] = df['QA'].map(lambda x:substring(x))

In [ ]:
df.tail()

In [ ]:
#DROP TWO COLUMNS ID AND TAG
df = df.drop(["id","tag"], axis = 1)
df.head()

In [ ]:
df.to_csv("Dataset/QA.txt", header = False, encoding = 'utf-8', index = None, mode ="w")

# QUESTION TEST PRE - PROCESSING

In [ ]:
df2 = pd.read_csv("Dataset/test-question.csv", sep = ";", encoding = 'utf8')
df2.head()

In [ ]:
df2 = df2.drop(["id"], axis = 1)
df2.head()

# TOKENIZER

In [ ]:
def fileCreator(filename, data):
    f = open(filename, "w")
    f.write(data)
    f.close()

In [ ]:
def callAPI(filename, token, email):
    f = open(filename)
    data = {'token': token, 'service':'tok', 'email':email, 'authentication':'google', 'format':'plain'}
    r = requests.post(url='http://tanl.di.unipi.it/it/api', data=data, files = {'file':f})
    f.close()
    out = open(filename, "w")
    #print (r.text.encode('utf-8'))
    out.write(r.text)
    out.close()

In [ ]:
def processingResponse(filename, gen):
    with open(filename) as f:
        line = f.readline()
        ar = []
        while line:
            print(line)
            item = str(line)
            item = re.sub('\n$', '', item)
            item = re.sub('\.$', '', item)
            if (item != ''):
                ar.append(item)
            line = f.readline()
        gen.append(ar)
    f.close()

In [ ]:
def cleaning(gen):
    for elem in gen:
        for e in elem:
            if(e == ""):
                elem.remove(e)

In [ ]:
#http://tanl.di.unipi.it/it/overview.html
token = ''
email = ''

X = [] #Question
Y = [] #Answer
Z = [] #Question+Answer
T = [] #Test

full=[] #Question+Answer+QuestionTest

for (idx,row) in df2.iterrows():
#for (idx,row) in df.iterrows():
    #For each row Q+A
    #data1 = row.loc['answer']
    #data2 = row.loc['QA']
    #data3 = row.loc['question']
    dataTest = row.loc["question-test"]
    
    #create file for TANL
    fileCreator("toTokenizerQTest.txt", dataTest)
    #fileCreator("toTokenizerQA.txt", data2)
    #fileCreator("toTokenizerOnlyA.txt", data1)
    #fileCreator("toTokenizerOnlyQ.txt", data3)
    
    #call API - return tokenized
    #callAPI("toTokenizerQA.txt",token, email)
    #callAPI("toTokenizerOnlyA.txt",token, email)
    #callAPI("toTokenizerOnlyQ.txt",token, email)
    callAPI("toTokenizerQTest.txt",token, email)
    
    #response manipulate
    processingResponse("toTokenizerQTest.txt", T)
    #processingResponse("toTokenizerQA.txt", Z)
    #processingResponse("toTokenizerOnlyA.txt", Y)
    #processingResponse("toTokenizerOnlyQ.txt", X)
    #cleaning
    full.extend(Z)
    full.extend(T)
    #cleaning(X)
    #cleaning(Y)
    #cleaning(Z)
    cleaning(T)
    #Sleep to avoid block request
    print ("SLEEP")
    time.sleep(5)


In [ ]:
#QUESTIONS
with open("tokenizerQuestion.txt", "w") as f:
    for elem in X:
        f.write("[\n")
        for item in elem:
            item = str(item)
            f.write(item + "\n")
        print (elem)
        f.write("]\n")
    f.close()

In [ ]:
#ANSWERS
with open("tokenizerAnswer.txt", "w") as f:
    for elem in Y:
        f.write("[\n")
        for item in elem:
            item = str(item)
            f.write(item + "\n")
        print (elem)
        f.write("]\n")
    f.close()

In [ ]:
#QUESTIONS+ANSWERS
with open("tokenizerQuestionAnswer.txt", "w") as f:
    for elem in Z:
        f.write("[\n")
        for item in elem:
            item = str(item)
            f.write(item + "\n")
        print (elem)
        f.write("]\n")
    f.close()

In [ ]:
#QUESTIONS TEST
with open("tokenizerQTest.txt", "w") as f:
    for elem in T:
        f.write("[\n")
        for item in elem:
            item = str(item)
            f.write(item + "\n")
        print (elem)
        f.write("]\n")
    f.close()

# GENSIM W2V

In [ ]:
#Word to Vect
# import modules & set up logging
import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
 
# train word2vec 
iterN = 10 #how many times the training code will run through the data set to train 
#the neural network (kind of like the number of training epochs).
mc = 1 #specifies the minimum amount of times that the word has to appear in the corpus 
#before it is included in the vocabulary – this allows us to easily eliminate rare words 
#and reduce our vocabulary size
s = 200 #In other words, each word in our vocabulary, after training, will be represented 
#by a 200 length word vector.
job = 4 #parallel workers we would like to work on the data – this will speed up the training process
window = 5 #contorno di 5 parole (default)

#full
modelFull = gensim.models.Word2Vec(full, iter = iterN, min_count=mc, size = s)

In [ ]:
#save w2v model
modelFull.save('model/mymodelFull')
#load w2v model
#new_model = gensim.models.Word2Vec.load('/tmp/mymodel')

In [ ]:
modelFull.most_similar(['n335', 'posso', 'bolletta'])

In [ ]:
modelFull.wv.syn0[1]

In [ ]:
# some similarity fun
print(modelFull.wv.similarity('come', 'bolletta'))

In [ ]:
# get the most common words
print(modelFull.wv.index2word[0], modelFull.wv.index2word[1], modelFull.wv.index2word[2])

In [ ]:
# get the least common words
vocab_size = len(modelFull.wv.vocab)
print(modelFull.wv.index2word[vocab_size - 1],modelFull.wv.index2word[vocab_size - 2], modelFull.wv.index2word[vocab_size - 3])